In [1]:
# Load AIEEE Data

In [1]:
!nvidia-smi

Fri Apr  1 11:21:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   49C    P0   192W / 250W |   7327MiB / 16280MiB |     93%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"    
import tensorflow as tf

In [2]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [3]:
import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

In [4]:
er_df = pd.read_csv("Models/Data/ERData/er_data_with_majority_label.csv")
er_df = er_df.dropna()

In [5]:
er_df.shape

(7149488, 3)

Load Models

In [6]:
#  other models
AIEEE_BERT = "Models/AIEEEGenderData/BERT/SavedModel/BERT"
CBSE_BERT = "Models/CBSEData/BERT/SavedModel/BERT"
ER_BERT = "Models/ERData/BERT/SavedModel/BERT"

In [7]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [8]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [10]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_BERT)

In [9]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_BERT)

In [ ]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_BERT)

In [10]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


Predict Gender

In [11]:
from transformers import pipeline

In [12]:
from sklearn.metrics import classification_report

# print(classification_report(true, labels, digits = 4))

In [13]:
!pip install datasets

In [19]:
er_df.head()

,Name,Gender
0,patel vinaben kanubhai,0.0
1,bankim kantibhai patel,1.0
2,kamli mukesh rameshchandra,1.0
3,devisharan babulal yogi,1.0
4,anita devisharan yogi,0.0


In [18]:
del er_df['Unnamed: 0']

In [22]:
from datasets import Dataset
import pandas as pd

dataset = Dataset.from_pandas(er_df[:1000])

In [23]:
dataset

Dataset({
    features: ['Name', 'Gender', '__index_level_0__'],
    num_rows: 1000
})

In [15]:
er_df.shape

(7149488, 3)

In [33]:
import math

def print_results(rmodel, tokenizer, df):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer, device=0)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
        with open("Models/ERData/er_data_cbse_bert_test1.txt", 'a') as f:
            f.write(f"{i}\n")
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    report = classification_report(true, labels, digits = 4)
    print(classification_report(true, labels, digits = 4))
    with open("Models/ERData/er_data_cbse_bert_test1.txt", 'a') as f:
            f.write(f"{report}\n")
    return report

In [34]:
er_df.shape

(7149488, 2)

In [35]:
report1 = print_results(cbse_model, tokenizer, er_df)

              precision    recall  f1-score   support

         0.0     0.7853    0.7363    0.7601   3336462
         1.0     0.7812    0.8239    0.8020   3813026

    accuracy                         0.7830   7149488
   macro avg     0.7833    0.7801    0.7810   7149488
weighted avg     0.7832    0.7830    0.7824   7149488



In [ ]:
report2 = print_results(cbse_model, tokenizer, er_df[2000000:4000000])

In [ ]:
report3 = print_results(cbse_model, tokenizer, er_df[4000000:6000000])

In [ ]:
report4 = print_results(cbse_model, tokenizer, er_df[6000000:])